In [2]:
import quantities as pq

In [3]:
zh2en_unit = {'公尺': 'meter', '公里': 'km'}


In [4]:
q = 2 * (pq.meter)**2

In [5]:
q.units = (pq.km) **2

In [10]:
q.array

AttributeError: 'Quantity' object has no attribute 'array'

In [7]:
dir(q)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__r

In [17]:
from stanfordnlp.server import CoreNLPClient
import re
# fix bug: google.protobuf.message.DecodeError: Error parsing message
# ref: https://github.com/stanfordnlp/stanfordnlp/issues/154
from google.protobuf.pyext._message import SetAllowOversizeProtos
SetAllowOversizeProtos(True)

nlp = CoreNLPClient(endpoint='http://140.109.19.190:9000', annotators="tokenize,ssplit,lemma,pos,ner",
							start_server=False, properties='chinese')

def get_mentions_from_text(text):
	nlp_data = nlp.annotate(text, properties={'pipelineLanguage': 'zh', 'annotators': "tokenize,lemma,pos,ner"})
	for mention in nlp_data.mentions:
		# def tok_ix_2_char_ix
	
		all_tokens = [token for sent in nlp_data.sentence for token in sent.token]
		len_all_tokens = list(filter(bool, [len(token.originalText) for token in all_tokens]))
	
		assert len(len_all_tokens) == len(all_tokens)
		s = 0
		tok_char_indexes = [s]
		for l in len_all_tokens:
			s += l
			tok_char_indexes.append(s)
	
		# solve the problem that tokenization does not include newlines
		for token in all_tokens:
			tok_ix = token.tokenBeginIndex
			char_ix = tok_char_indexes[tok_ix]
			text_by_char_ix = nlp_data.text[char_ix: char_ix + len(token.originalText)]
			newlines = re.findall('[\n\s\t]', text_by_char_ix)
			if newlines:
				for ix in range(tok_ix, len(all_tokens)):
					tok_char_indexes[ix] += len(newlines)
			# for debugging
			# tok_ix = token.tokenBeginIndex
			# char_ix = tok_char_indexes[tok_ix]
			# print(tok_ix, char_ix, token.originalText, passage_data.text[char_ix: char_ix + len(token.originalText)])
		
		# mention_slice = (tok_char_indexes[mention.tokenStartInSentenceInclusive], tok_char_indexes[mention.tokenEndInSentenceExclusive])
		char_b, char_e = tok_char_indexes[mention.tokenStartInSentenceInclusive], tok_char_indexes[mention.tokenEndInSentenceExclusive]
		yield {'text': mention.entityMentionText, 'char_b': char_b, 'char_e': char_e, 'ner': mention.ner}

list(get_mentions_from_text('苏东坡在中国历史上，是哪一个朝代的人？'))

[{'text': '苏东坡', 'char_b': 0, 'char_e': 3, 'ner': 'PERSON'},
 {'text': '中国', 'char_b': 4, 'char_e': 6, 'ner': 'COUNTRY'},
 {'text': '一', 'char_b': 12, 'char_e': 13, 'ner': 'NUMBER'}]